In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import popoff
from Session import SessionLite, build_flu_array_single
from mpl_toolkits.mplot3d import Axes3D
from linear_model import PoolAcrossSessions, LinearModel, pca_session, LabelEncoder, largest_PC_trace, largest_PC_loading, do_pca
import loadpaths
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys, os, pickle, copy
from sklearn.decomposition import PCA
import warnings
import pandas as pd
import math, cmath
from tqdm import tqdm
import pop_off_functions as pof
import pop_off_plotting as pop
from IPython.core.debugger import Pdb
ipdb = Pdb()

# sys.path.append(os.path.expanduser('~/Documents/code/PtitPrince/pititprince'))
import ptitprince as pt

## Add paths:
# user_paths_dict = loadpaths.loadpaths()

# path_to_vape = user_paths_dict['vape_path']

# sys.path.append(str(path_to_vape))
# sys.path.append(str(os.path.join(path_to_vape, 'jupyter')))
# sys.path.append(str(os.path.join(path_to_vape, 'utils')))

# user_paths = loadpaths.loadpaths()
# sys.path.append(user_paths['base_path'])
# figure_path = '/home/tplas/repos/optistim_results/'
# from pySpatialNeuronNet import factor_analysis


# COLORS = [
# '#228E5E', # Hit green
# '#DB3713', # Miss red
# ]


## Load data in form of Jimmy's PAS object
warnings.warn('Targets in data, fix self.ever_targetted')
remove_targets = False
pas = PoolAcrossSessions(save_PCA=False, subsample_sessions=False,
                         remove_targets=remove_targets, remove_toosoon = True)
lm = pas.linear_models[0]
print(pas.sessions)

In [ ]:
## Create sessions object from PAS:

try:  # ensure sessions doesn't exist yet 
    sessions
    assert type(sessions) is dict
except NameError:
    pass

sessions = {}
int_keys_pas_sessions = pas.sessions.keys()
print(int_keys_pas_sessions)
i_s = 0
for ses in pas.sessions.values():  # load into sessions dict (in case pas skips an int as key)
    ses.signature = f'{ses.mouse}_R{ses.run_number}'
    sessions[i_s] = ses
    i_s += 1
#     if ses.signature != 'RL117_R29':  # one that used not to have spont data
#         sessions[i_s] = ses
#         i_s += 1
#     else:
#         f'{ses.signature} has been excluded !!! (because no spont data available)'
print(sessions)
assert len(sessions) == 10
pof.label_urh_arm(sessions=sessions)  # label arm and urh

# for kk, ss in sessions.items():
    
    
print('------------------------------------')
print(f'{len(sessions)} sessions are loaded')
tp_dict = pof.create_tp_dict(sessions=sessions)
tp_dict['decoders'] = tp_dict['mutual']#[np.logical_and(tp_dict['mutual'] >-2]#, tp_dict['mutual'] <= 4)]


## Train decoders (takes 20 mins per decoder type)

In [ ]:
## Compute results decoders (note: CV of regularisation is down below in the notebook)
(lick_acc, lick_acc_split, lick_pred_split, ps_acc, ps_acc_split, ps_pred_split, lick_half, 
     angle_dec, decoder_weights) = pof.compute_accuracy_time_array_average_per_mouse(sessions=sessions, 
                                               time_array=tp_dict['decoders'][40:280],
                                              projected_data=False, 
                                              reg_type='l2', regularizer=1,  #reg_type='none',
                                              average_fun=pof.class_av_mean_accuracy,
                                              split_fourway=True,
                                                list_tt_training=['cr', 'spont'])

In [ ]:
## Spont used for training

plot_tt = ['hit', 'spont', 'miss', 'fp', 'cr']
plot_dict_split = {x: ps_pred_split[x] for x in plot_tt}
pop.plot_dynamic_decoding_two_regions(ps_acc_split=plot_dict_split,
                                      time_array=tp_dict['decoders'][40:280],
                                      yaxis_type='prediction', 
                                      smooth_traces=True,
                                      one_sided_window_size=2,
                                      save_fig=False,
                                      fn_suffix='subsampled_SpontCr_10-sessions_ws2',
                                      plot_indiv=False,
                                      plot_std_area=True)